Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [3]:
pickle_file = 'notMNIST_sanitized.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
#   valid_dataset = save['sanitized_valid_dataset']
#   valid_labels = save['sanitized_valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
#   print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (187196, 28, 28) (187196,)
Test set (8697, 28, 28) (8697,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [4]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
# valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
# print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (187196, 784) (187196, 10)
Test set (8697, 784) (8697, 10)


In [5]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [89]:
batch = 128
hl1 = 1024 # No. of hidden units
num_classes = 10

from math import sqrt

graph = tf.Graph()
with graph.as_default():

  tf_train_dataset = tf.placeholder(tf.float32,shape=(None, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(None, num_classes))
  
  
  tf_test_dataset = tf.constant(test_dataset)
  tf_regularization_parameter = 0.001
  
  
  hidden_weights1 = tf.Variable(tf.truncated_normal([image_size * image_size, hl1],stddev=sqrt(2.0/(image_size*image_size + hl1))))
  hidden_biases1 = tf.Variable(tf.zeros([hl1]))
  output_weights = tf.Variable(tf.truncated_normal([hl1, num_classes],stddev=sqrt(2.0/(hl1 + num_classes))))
  output_biases = tf.Variable(tf.zeros([num_classes]))
  
  
  l1 = tf.nn.relu(tf.matmul(tf_train_dataset, hidden_weights1) + hidden_biases1)
  output = tf.matmul(l1, output_weights) + output_biases
    
  regularizer = tf.nn.l2_loss(hidden_weights1) + tf.nn.l2_loss(hidden_biases1) + tf.nn.l2_loss(output_weights) + tf.nn.l2_loss(output_biases)
  
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(output, tf_train_labels))
  l2_loss = tf.reduce_mean(loss + (tf_regularization_parameter * regularizer))
    
  optimizer = tf.train.MomentumOptimizer(0.03, 0.5).minimize(l2_loss)
  
  
  train_prediction = tf.nn.softmax(output)
  h1_test = tf.nn.relu(tf.matmul(tf_test_dataset, hidden_weights1) + hidden_biases1)
  test_output = tf.matmul(h1_test, output_weights) + output_biases

  test_prediction = tf.nn.softmax(test_output)

In [10]:
hm_epochs = 10
train_batch_size = train_dataset.shape[0]

with tf.Session(graph=graph) as session:
  session.run(tf.initialize_all_variables())

  for epoch in range(hm_epochs):
    epoch_loss = 0
    i=0
    while i < train_batch_size:
      start = i
      end = i+batch
      batch_x = np.array(train_dataset[start:end])
      batch_y = np.array(train_labels[start:end])
      feed_dict = {tf_train_dataset : batch_x, tf_train_labels : batch_y}
      _, l, predictions = session.run([optimizer, l2_loss, train_prediction], feed_dict=feed_dict)
      epoch_loss += l
      i+=batch
    print('Epoch', epoch, 'completed out of', hm_epochs, 'loss', epoch_loss)
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Epoch 0 completed out of 10 loss 1370.9301374
Epoch 1 completed out of 10 loss 1146.43448406
Epoch 2 completed out of 10 loss 1035.38426876
Epoch 3 completed out of 10 loss 952.956896722
Epoch 4 completed out of 10 loss 889.222707897
Epoch 5 completed out of 10 loss 838.622138649
Epoch 6 completed out of 10 loss 797.766390324
Epoch 7 completed out of 10 loss 764.447009146
Epoch 8 completed out of 10 loss 737.037339032
Epoch 9 completed out of 10 loss 714.340242624
Test accuracy: 94.3%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [90]:
hm_epochs = 10
mini_train_batch_size = 5*batch

with tf.Session(graph=graph) as session:
  session.run(tf.initialize_all_variables())

  for epoch in range(hm_epochs):
    epoch_loss = 0
    i=0
    while i < mini_train_batch_size:
      start = i
      end = i+batch
      batch_x = np.array(train_dataset[start:end])
      batch_y = np.array(train_labels[start:end])
      feed_dict = {tf_train_dataset : batch_x, tf_train_labels : batch_y}
      _, l, predictions = session.run([optimizer, l2_loss, train_prediction], feed_dict=feed_dict)
      epoch_loss += l
      i+=batch
    print('Epoch', epoch, 'completed out of', hm_epochs, 'loss', epoch_loss)
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))


Epoch 0 completed out of 10 loss 12.6356585026
Epoch 1 completed out of 10 loss 10.0167533159
Epoch 2 completed out of 10 loss 8.36176371574
Epoch 3 completed out of 10 loss 7.35143589973
Epoch 4 completed out of 10 loss 6.68097102642
Epoch 5 completed out of 10 loss 6.20217561722
Epoch 6 completed out of 10 loss 5.84070634842
Epoch 7 completed out of 10 loss 5.55540680885
Epoch 8 completed out of 10 loss 5.32211959362
Epoch 9 completed out of 10 loss 5.12566608191
Test accuracy: 82.9%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [86]:
batch = 128
hl1 = 1024 # No. of hidden units
num_classes = 10

from math import sqrt

graph = tf.Graph()
with graph.as_default():

  tf_train_dataset = tf.placeholder(tf.float32,shape=(None, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(None, num_classes))
  
  
  tf_test_dataset = tf.constant(test_dataset)
  tf_regularization_parameter = 0.001
  tf_dropout = 0.5
  
  
  hidden_weights1 = tf.Variable(tf.truncated_normal([image_size * image_size, hl1],stddev=sqrt(2.0/(image_size*image_size + hl1))))
  hidden_biases1 = tf.Variable(tf.zeros([hl1]))
  output_weights = tf.Variable(tf.truncated_normal([hl1, num_classes],stddev=sqrt(2.0/(hl1 + num_classes))))
  output_biases = tf.Variable(tf.zeros([num_classes]))
  
  
  l1 = tf.nn.relu(tf.matmul(tf_train_dataset, hidden_weights1) + hidden_biases1)
  l1 = tf.nn.dropout(l1,tf_dropout)  
  output = tf.matmul(l1, output_weights) + output_biases
    
  regularizer = tf.nn.l2_loss(hidden_weights1) + tf.nn.l2_loss(hidden_biases1) + tf.nn.l2_loss(output_weights) + tf.nn.l2_loss(output_biases)
  
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(output, tf_train_labels))
  l2_loss = tf.reduce_mean(loss + (tf_regularization_parameter * regularizer))
    
  optimizer = tf.train.MomentumOptimizer(0.03, 0.5).minimize(l2_loss)
  
  
  train_prediction = tf.nn.softmax(output)
  h1_test = tf.nn.relu(tf.matmul(tf_test_dataset, hidden_weights1) + hidden_biases1)
  test_output = tf.matmul(h1_test, output_weights) + output_biases

  test_prediction = tf.nn.softmax(test_output)

In [52]:
#################################3
# Multilayer perceptron with Dropout
#################################3

batch = 128
hl1 = 1024 # No. of hidden units
num_classes = 10

from math import sqrt

graphML_with_dropout = tf.Graph()
with graphML_with_dropout.as_default():

  tf_train_dataset = tf.placeholder(tf.float32,shape=(None, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(None, num_classes))
  
#   tf_valid_dataset = tf.constant(valid_dataset)
  
  tf_test_dataset = tf.constant(test_dataset)
  tf_regularization_parameter = 0.001
  tf_dropout = 0.5
  

  
  hidden_weights1 = tf.Variable(tf.truncated_normal([image_size * image_size, hl1],stddev=sqrt(2.0/(image_size*image_size + hl1))))
  hidden_biases1 = tf.Variable(tf.zeros([hl1]))
  hidden_weights2 = tf.Variable(tf.truncated_normal([hl1, hl1],stddev=sqrt(2.0/(hl1 + hl1))))
  hidden_biases2 = tf.Variable(tf.zeros([hl1]))
  output_weights = tf.Variable(tf.truncated_normal([hl1, num_classes],stddev=sqrt(2.0/(hl1 + num_classes))))
  output_biases = tf.Variable(tf.zeros([num_classes]))
  
  
  l1 = tf.nn.relu(tf.matmul(tf_train_dataset, hidden_weights1) + hidden_biases1)
  l1 = tf.nn.dropout(l1,tf_dropout)
  l2 = tf.nn.relu(tf.matmul(l1,hidden_weights2) + hidden_biases2)
  l2 = tf.nn.dropout(l2, tf_dropout)
  output = tf.matmul(l2, output_weights) + output_biases
    
  regularizer = tf.nn.l2_loss(hidden_weights1) + tf.nn.l2_loss(hidden_weights2) + tf.nn.l2_loss(output_weights)
  
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(output, tf_train_labels))
  l2_loss = tf.reduce_mean(loss + (tf_regularization_parameter * regularizer))
  
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(0.5,
                                               global_step, 
                                               decay_steps=50000,
                                               decay_rate=0.001)

  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(l2_loss, global_step = global_step)
  
  
  train_prediction = tf.nn.softmax(output)
  h1_test = tf.nn.relu(tf.matmul(tf_test_dataset, hidden_weights1) + hidden_biases1)
  h1_test = tf.nn.dropout(h1_test, tf_dropout)
  test_output = tf.matmul(h1_test, output_weights) + output_biases

  test_prediction = tf.nn.softmax(test_output)

In [88]:
hm_epochs = 10

with tf.Session(graph=graph) as session:
  session.run(tf.initialize_all_variables())

  for epoch in range(hm_epochs):
    epoch_loss = 0
    i=0
    while i < mini_train_batch_size:
      start = i
      end = i+batch
      batch_x = np.array(mini_train_data[start:end])
      batch_y = np.array(mini_train_labels[start:end])
      feed_dict = {tf_train_dataset : batch_x, tf_train_labels : batch_y}
      _, l, predictions = session.run([optimizer, l2_loss, train_prediction], feed_dict=feed_dict)
      epoch_loss += l
      i+=batch
    print('Epoch', epoch, 'completed out of', hm_epochs, 'loss', epoch_loss)
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Epoch 0 completed out of 10 loss 13.1265692711
Epoch 1 completed out of 10 loss 10.4547333717
Epoch 2 completed out of 10 loss 8.86869597435
Epoch 3 completed out of 10 loss 7.69887089729
Epoch 4 completed out of 10 loss 7.03619945049
Epoch 5 completed out of 10 loss 6.63552606106
Epoch 6 completed out of 10 loss 6.22493517399
Epoch 7 completed out of 10 loss 5.91891276836
Epoch 8 completed out of 10 loss 5.65975475311
Epoch 9 completed out of 10 loss 5.44233059883
Test accuracy: 82.6%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [94]:
batch = 128
hl1 = 1024 # No. of hidden units
num_classes = 10

from math import sqrt

graphNN = tf.Graph()
with graphNN.as_default():

  tf_train_dataset = tf.placeholder(tf.float32,shape=(None, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(None, num_classes))
  
  
  tf_test_dataset = tf.constant(test_dataset)
  tf_regularization_parameter = 0.001
  tf_dropout = 0.5
  

  
  hidden_weights1 = tf.Variable(tf.truncated_normal([image_size * image_size, hl1],stddev=sqrt(2.0/(image_size*image_size + hl1))))
  hidden_weights2 = tf.Variable(tf.truncated_normal([hl1, hl1], stddev=sqrt(2.0/(image_size*image_size + hl1))))
  output_weights = tf.Variable(tf.truncated_normal([hl1, num_classes]))
  hidden_biases1 = tf.Variable(tf.zeros([hl1]))
  hidden_biases2 = tf.Variable(tf.zeros([hl1]))
  output_biases = tf.Variable(tf.zeros([num_classes]))
  
  l1 = tf.nn.relu(tf.matmul(tf_train_dataset, hidden_weights1) + hidden_biases1) 
  l2 = tf.nn.relu(tf.matmul(l1, hidden_weights2) + hidden_biases2)

  output = tf.matmul(l2, output_weights) + output_biases
    
  regularizer = tf.nn.l2_loss(hidden_weights1) + tf.nn.l2_loss(hidden_biases1) + tf.nn.l2_loss(output_weights) + tf.nn.l2_loss(output_biases)
  
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(output, tf_train_labels))
#   l2_loss = tf.reduce_mean(loss + (tf_regularization_parameter * regularizer))

#   global_step = tf.Variable(0)
#   learning_rate = tf.train.exponential_decay(0.5,
#                                                global_step, 
#                                                decay_steps=10000,
#                                                decay_rate=0.001)
  optimizer = tf.train.GradientDescentOptimizer(0.03).minimize(loss)
  
  train_prediction = tf.nn.softmax(output)
  h1_test = tf.nn.relu(tf.matmul(tf_test_dataset, hidden_weights1) + hidden_biases1)
  h2_test = tf.nn.relu(tf.matmul(h1_test, hidden_weights2) + hidden_biases2)
    
  test_output = tf.matmul(h2_test, output_weights) + output_biases

  test_prediction = tf.nn.softmax(test_output)

In [95]:
hm_epochs = 10
with tf.Session(graph=graphNN) as session:
  session.run(tf.initialize_all_variables())

  for epoch in range(hm_epochs):
#     print(learning_rate.eval())
    epoch_loss = 0
    i=0
    while i < train_dataset.shape[0]:
      start = i
      end = i+batch
      batch_x = np.array(train_dataset[start:end])
      batch_y = np.array(train_labels[start:end])
      feed_dict = {tf_train_dataset : batch_x, tf_train_labels : batch_y}
      _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
      epoch_loss += l
      i+=batch
    print('Epoch', epoch, 'completed out of', hm_epochs, 'loss', epoch_loss)
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Epoch 0 completed out of 10 loss 793.307695925
Epoch 1 completed out of 10 loss 459.092164293
Epoch 2 completed out of 10 loss 349.656610101
Epoch 3 completed out of 10 loss 272.254500035
Epoch 4 completed out of 10 loss 214.366618596
Epoch 5 completed out of 10 loss 169.574574228
Epoch 6 completed out of 10 loss 135.150065487
Epoch 7 completed out of 10 loss 109.477399016
Epoch 8 completed out of 10 loss 89.6907814648
Epoch 9 completed out of 10 loss 74.7751366831
Test accuracy: 95.3%
